In [ ]:

library(tidyverse) # metapackage of all tidyverse packages

options(warn = -1, repr.plot.width = 14, repr.plot.height =  10)

fig <- function(width, heigth){
    options(repr.plot.width = width, repr.plot.height = heigth)
}

set.seed(228)

files = list.files(path = "../input/fashionmnist")

files = paste0('../input/fashionmnist/',files)

files

# Read data and prepair

## Reading

In [ ]:
train = read_csv(files[2])
test = read_csv(files[1])

head(train)

head(test)


In [ ]:
dim(train)

dim(test)

## Scale data

In [ ]:
train[,1:784 + 1] = train[,1:784 + 1] / 256

test[,1:784 + 1] = test[,1:784 + 1] / 256

head(train)

## Train, validation and test datasets

In [ ]:
cols = 1:784+1

X_test = as.matrix(test[,cols])
y_test = test$label

y_test[1:10]

In [ ]:
tr = sample(1:60000, 60000*0.8)

X_train = as.matrix(train[tr, cols])
y_train = train$label[tr]

X_valid = as.matrix(train[-tr, cols])
y_valid = train$label[-tr]

dim(X_train)

dim(X_valid)

# Models

## Keras preparations

In [ ]:
library(keras)

y_test = to_categorical(y_test, num_classes = 10)
y_train = to_categorical(y_train, num_classes = 10)
y_valid = to_categorical(y_valid, num_classes = 10)


In [ ]:
dim(y_train)

## Logistic regression

In [ ]:
model = keras_model_sequential() %>%
            layer_dense(input_shape = c(784), units = 10, activation = 'softmax')

model %>% compile(
            optimizer = 'sgd',
            loss = 'categorical_crossentropy',
            metrics = 'accuracy'
        )

summary(model)

In [ ]:
history = model %>% fit(
         x = X_train,
        y = y_train,
    batch_size = 128,
    epochs = 100,
    validation_data = list(X_valid, y_valid),
    verbose = TRUE
    )

plot(history)

No overfitting

Train on all train data and evaluate on test:

In [ ]:
model %>% fit(
         x = rbind(X_train, X_valid),
        y = rbind(y_train, y_valid),
    batch_size = 128,
    epochs = 100
    )

model %>% evaluate(X_test, y_test)


# Dense model

In [ ]:
model = keras_model_sequential() %>%
            layer_dense(input_shape = c(784), units = 256, activation = 'relu') %>%
            layer_dense( units = 10, activation = 'softmax')

model %>% compile(
            optimizer = 'sgd',
            loss = 'categorical_crossentropy',
            metrics = 'accuracy'
        )

summary(model)

In [ ]:
history = model %>% fit(
         x = X_train,
        y = y_train,
    batch_size = 128,
    epochs = 100,
    validation_data = list(X_valid, y_valid),
    verbose = TRUE
    )

plot(history)

Better quality. Add dropout:

In [ ]:
model = keras_model_sequential() %>%
            layer_dense(input_shape = c(784), units = 256, activation = 'relu') %>%
            layer_dropout(0.4) %>%
            layer_dense( units = 10, activation = 'softmax')

model %>% compile(
            optimizer = 'sgd',
            loss = 'categorical_crossentropy',
            metrics = 'accuracy'
        )

history = model %>% fit(
         x = X_train,
        y = y_train,
    batch_size = 128,
    epochs = 100,
    validation_data = list(X_valid, y_valid),
    verbose = TRUE
    )

plot(history)

Not better. Add new layer:

In [ ]:
model = keras_model_sequential() %>%
            layer_dense(input_shape = c(784), units = 256, activation = 'relu') %>%
            layer_dropout(0.4) %>%
            layer_dense(units = 128, activation = 'relu') %>%
            layer_dropout(0.5) %>%
            layer_dense(units = 10, activation = 'softmax')

model %>% compile(
            optimizer = 'sgd',
            loss = 'categorical_crossentropy',
            metrics = 'accuracy'
        )

history = model %>% fit(
         x = X_train,
        y = y_train,
    batch_size = 128,
    epochs = 100,
    validation_data = list(X_valid, y_valid),
    verbose = TRUE
    )

plot(history)

Not better. I will use old configuration:

In [ ]:
model = keras_model_sequential() %>%
            layer_dense(input_shape = c(784), units = 256, activation = 'relu') %>%
            layer_dense( units = 10, activation = 'softmax')

model %>% compile(
            optimizer = 'sgd',
            loss = 'categorical_crossentropy',
            metrics = 'accuracy'
        )

model %>% fit(
         x = rbind(X_train, X_valid),
        y = rbind(y_train, y_valid),
    batch_size = 128,
    epochs = 100
    )

model %>% evaluate(X_test, y_test)


## Convolution model

Reshape data:

In [ ]:
x_train = array_reshape(X_train, dim = c(dim(X_train)[1], 28, 28, 1))
x_valid = array_reshape(X_valid, dim = c(dim(X_valid)[1], 28, 28, 1))
x_test = array_reshape(X_test, dim = c(dim(X_test)[1], 28, 28, 1))

Simple model:

In [ ]:
model = keras_model_sequential() %>%
            layer_conv_2d(input_shape = c(28,28,1), filters = 32, kernel_size = c(3,3), activation = 'relu') %>%
            layer_max_pooling_2d(pool_size = c(2,2)) %>%
            layer_conv_2d(filters = 64, kernel_size = c(3,3), activation = 'relu') %>%
            layer_max_pooling_2d(pool_size = c(2,2)) %>%
            layer_flatten() %>%
            layer_dense( units = 10, activation = 'softmax')

model %>% compile(
            optimizer = 'adam',
            loss = 'categorical_crossentropy',
            metrics = 'accuracy'
        )

summary(model)

In [ ]:

history = model %>% fit(
         x = x_train,
        y = y_train,
    batch_size = 128,
    epochs = 25,
    validation_data = list(x_valid, y_valid),
    verbose = TRUE
    )

plot(history)

In [ ]:
model %>% evaluate(x_test, y_test)

Early overfitting. Add dropout:

In [ ]:
model = keras_model_sequential() %>%
            layer_conv_2d(input_shape = c(28,28,1), filters = 32, kernel_size = c(3,3), activation = 'relu') %>%
            layer_max_pooling_2d(pool_size = c(2,2)) %>%
            layer_conv_2d(filters = 64, kernel_size = c(2,2), activation = 'relu') %>%
            layer_max_pooling_2d(pool_size = c(2,2)) %>%
            layer_dropout(0.5) %>%
            layer_flatten() %>%
            layer_dense( units = 10, activation = 'softmax')

model %>% compile(
            optimizer = 'rmsprop',
            loss = 'categorical_crossentropy',
            metrics = 'accuracy'
        )

history = model %>% fit(
         x = x_train,
        y = y_train,
    batch_size = 128,
    epochs = 25,
    validation_data = list(x_valid, y_valid),
    verbose = TRUE
    )

plot(history)

In [ ]:
model %>% evaluate(x_test, y_test)

add batch normalization

In [ ]:
model = keras_model_sequential() %>%
            layer_conv_2d(input_shape = c(28,28,1), filters = 32, kernel_size = c(3,3), activation = 'relu') %>%
            layer_max_pooling_2d(pool_size = c(2,2)) %>%
            layer_batch_normalization() %>%
            layer_conv_2d(filters = 64, kernel_size = c(2,2), activation = 'relu') %>%
            layer_max_pooling_2d(pool_size = c(2,2)) %>%
            layer_batch_normalization() %>%
            layer_dropout(0.5) %>%
            layer_flatten() %>%
            layer_dense( units = 10, activation = 'softmax')

model %>% compile(
            optimizer = 'rmsprop',
            loss = 'categorical_crossentropy',
            metrics = 'accuracy'
        )

history = model %>% fit(
         x = x_train,
        y = y_train,
    batch_size = 128,
    epochs = 25,
    validation_data = list(x_valid, y_valid),
    verbose = TRUE
    )

plot(history)

In [ ]:
model %>% evaluate(x_test, y_test)